In [1]:
import sys
import os

original_sys_path = sys.path.copy()
try:
    sys.path.insert(0, os.path.abspath(".."))
    from src.modeling_helpers import load_trial_data
finally:
    sys.path = original_sys_path

# Load data
path_to_data = "../data/classification/smiles/full_clean"
train_df, val_df, test_df = load_trial_data(path_to_data)

from sklearn.metrics import f1_score, balanced_accuracy_score
import numpy as np
import pandas as pd


def compute_baseline_test_performance_(
    train_df: pd.DataFrame, test_df: pd.DataFrame
) -> (float, float, float):
    """
    Computes the baseline test performance using the majority class from the training set and returns
    the micro F1 score, the balanced accuracy score, and the accuracy score of the baseline classifier.

    Args:
        train_df (pd.DataFrame): The training dataset containing labels.
        test_df (pd.DataFrame): The test dataset containing labels.

    Returns:
        float: The micro F1 score of the baseline classifier.
        float: The balanced accuracy score of the baseline classifier.
        float: The accuracy score of the baseline classifier.
    """
    # Extract actual labels from the test set
    actuals = test_df.drop(
        columns=["group_description", "eligibility_criteria", "smiles"]
    ).values

    # Calculate the majority class from the training set
    train_df_label = train_df.drop(
        columns=["group_description", "eligibility_criteria", "smiles"]
    )
    majority_classes = (train_df_label.mean() > 0.5).astype(int).values

    # Create predictions using majority classes
    predictions_majority_class = np.tile(majority_classes, (actuals.shape[0], 1))

    # Compute the micro F1 score
    f1_score_majority_class = f1_score(
        actuals, predictions_majority_class, average="micro"
    )

    # Compute the balanced accuracy score
    balanced_accuracy_majority_class = balanced_accuracy_score(
        actuals.ravel(), predictions_majority_class.ravel()
    )

    return f1_score_majority_class, balanced_accuracy_majority_class


# Example usage
(
    f1_score_majority_class,
    balanced_accuracy_majority_class,
) = compute_baseline_test_performance_(train_df, test_df)

f1_score_majority_class, balanced_accuracy_majority_class

(0.38316613653046244, 0.6083015275720578)